## Create Test data for the emergency scenerio

In [2]:
import os
os.chdir('../..')
from trips.config import *
from trips.weibull_trips import main
import subprocess

In [ ]:
number_cars = 400

if not os.path.exists("test"):
    os.makedirs("test")
    

main(
    src_nodes,
    dst_nodes,
    src_prob,
    turn_prob,
    emergency_probs,
    shape,
    number_cars,
    total_timesteps,
)
subprocess.run(
    [
        "duarouter",
        "-n",
        "./network_details/intersection.net.xml",  # Input network file
        "-t",
        "trips.trips.xml",  # Input trips file
        "-o",
        f"test/intersection_Emergency.rou.xml",  # Output routes file
    ]
)
os.remove("trips.trips.xml")


route_files = ["test/intersection_Emergency.rou.xml"]
        

Success.up to time step: 5311.00


 Use 'route-files' instead.


In [3]:
from environment.environment import EmergencySumoEnvironment
from environment.observation import EmergencyObservationFunction
from environment.reward import emergency_reward

## Test round robin algorithm on emergency scenerio

In [ ]:
env = EmergencySumoEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/test/Emergency/test/RoundRobin',
    min_green=15,
    yellow_time=5,
    delta_time=20,
    use_gui=False,
    num_seconds=5400,
    observation_class=EmergencyObservationFunction,
    reward_fn=emergency_reward
)

obs, info = env.reset()
done = False

action_index = 0  
while not done:
    action = action_index % 4
    
    next_obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    action_index += 1

obs, info = env.reset()
env.close()

## Test the trained DQN policy on emergency scenerio

In [ ]:
from stable_baselines3 import DQN

model = DQN.load("agents/dqn_emergency")

env = EmergencySumoEnvironment(
    net_file="network_details/intersection.net.xml",
    route_files=route_files,
    out_csv_name=f'outputs/test/Emergency/DQN',
    min_green=5,
    yellow_time=5,
    delta_time=10,
    use_gui=False,
    num_seconds=5400,
    observation_class=EmergencyObservationFunction,
    reward_fn=emergency_reward
)

obs, info = env.reset()
done = False

while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

obs, info = env.reset()
env.close()